In [1]:
import librosa
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint 
from datetime import datetime
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

# seed = 42
# tf.random.set_seed(seed)
# np.random.seed(seed)

In [35]:
file_path = '/Users/atrix/Documents/desktop_folders/Programming/tensorflow/archive/fold'
urbansound8k = pd.read_csv("/Users/atrix/Documents/desktop_folders/Programming/tensorflow/archive/UrbanSound8K.csv")

In [30]:
urbansound8k.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [31]:
class_map = {'0' : 'air_conditioner',
             '1' : 'car_horn',
             '2' : 'children_playing', 
             '3' : 'dog_bark', 
             '4' : 'drilling', 
             '5' : 'engine_idling', 
             '6' : 'gun_shot',
             '7' : 'jackhammer',
             '8' : 'siren',
             '9' : 'street_music'}
pd.DataFrame(sorted(list(class_map.items())))

,0,1
0,0,air_conditioner
1,1,car_horn
2,2,children_playing
3,3,dog_bark
4,4,drilling
5,5,engine_idling
6,6,gun_shot
7,7,jackhammer
8,8,siren
9,9,street_music


In [32]:
max_pad_len = 174

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [33]:
features = []

# Iterate through each sound file and extract the features 
for index, row in urbansound8k.iterrows():
    
    file_name = file_path+str(row["fold"])+'/'+str(row["slice_file_name"])
    
    class_label = row["classID"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

/Users/atrix/miniconda3/envs/tensorflow/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/Users/atrix/miniconda3/envs/tensorflow/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/Users/atrix/miniconda3/envs/tensorflow/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [36]:
featuresdf.head()

,feature,class_label
0,"[[-332.03876, -169.58778, -90.24683, -56.92341...",3
1,"[[-486.05615, -447.30463, -441.84772, -436.221...",2
2,"[[-492.76328, -465.86954, -473.96152, -480.512...",2
3,"[[-441.2638, -405.25677, -401.20792, -400.8395...",2
4,"[[-467.48068, -431.21393, -435.46893, -434.359...",2


In [37]:
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

In [38]:
x_train1 = x_train 
x_test1 = x_test
y_train1 = y_train
y_test1 = y_test
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

6985
1747
6985
1747


In [39]:
num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
print(x_train.shape)

num_labels = yy.shape[1]
filter_size = 3

(6985, 40, 174, 1)


In [40]:
model_relu = Sequential()
model_relu.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model_relu.add(MaxPooling2D(pool_size=(2,2)))
model_relu.add(Dropout(0.2))

model_relu.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model_relu.add(MaxPooling2D(pool_size=(2,2)))
model_relu.add(Dropout(0.2))

model_relu.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model_relu.add(MaxPooling2D(pool_size=(2,2)))
model_relu.add(Dropout(0.2))

model_relu.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model_relu.add(MaxPooling2D(pool_size=(2,2)))
model_relu.add(Dropout(0.2))

model_relu.add(GlobalAveragePooling2D())
model_relu.add(Flatten())
model_relu.add(Dense(num_labels, activation='softmax'))

In [41]:
model_relu.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [46]:
model_relu.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 39, 173, 16)       80        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 19, 86, 16)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 19, 86, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 18, 85, 32)        2080      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 9, 42, 32)        0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 9, 42, 32)        

In [43]:
num_epochs = 300
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='saved_models/cnn3.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

history_relu = model_relu.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
                              validation_data = (x_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/300
28/28 [==============================] - ETA: 0s - loss: 3.6622 - accuracy: 0.1635
Epoch 1: val_loss improved from inf to 2.17644, saving model to saved_models/cnn3.hdf5
28/28 [==============================] - 3s 78ms/step - loss: 3.6622 - accuracy: 0.1635 - val_loss: 2.1764 - val_accuracy: 0.1849
Epoch 2/300
27/28 [===========================>..] - ETA: 0s - loss: 1.9852 - accuracy: 0.2925
Epoch 2: val_loss improved from 2.17644 to 1.93965, saving model to saved_models/cnn3.hdf5
28/28 [==============================] - 1s 48ms/step - loss: 1.9834 - accuracy: 0.2928 - val_loss: 1.9397 - val_accuracy: 0.3744
Epoch 3/300
27/28 [===========================>..] - ETA: 0s - loss: 1.7595 - accuracy: 0.3753
Epoch 3: val_loss improved from 1.93965 to 1.80822, saving model to saved_models/cnn3.hdf5
28/28 [==============================] - 1s 48ms/step - loss: 1.7597 - accuracy: 0.3758 - val_loss: 1.8082 - val_accuracy: 0.4156
Epoch 4/300
27/28 [===========================>..] - ETA

28/28 [==============================] - 1s 47ms/step - loss: 0.6101 - accuracy: 0.7924 - val_loss: 0.8059 - val_accuracy: 0.7459
Epoch 28/300
27/28 [===========================>..] - ETA: 0s - loss: 0.5942 - accuracy: 0.7970
Epoch 28: val_loss improved from 0.80594 to 0.77022, saving model to saved_models/cnn3.hdf5
28/28 [==============================] - 1s 47ms/step - loss: 0.5950 - accuracy: 0.7967 - val_loss: 0.7702 - val_accuracy: 0.7624
Epoch 29/300
27/28 [===========================>..] - ETA: 0s - loss: 0.5872 - accuracy: 0.8037
Epoch 29: val_loss did not improve from 0.77022
28/28 [==============================] - 1s 46ms/step - loss: 0.5862 - accuracy: 0.8037 - val_loss: 0.8214 - val_accuracy: 0.7361
Epoch 30/300
28/28 [==============================] - ETA: 0s - loss: 0.5644 - accuracy: 0.8119
Epoch 30: val_loss improved from 0.77022 to 0.76042, saving model to saved_models/cnn3.hdf5
28/28 [==============================] - 1s 48ms/step - loss: 0.5644 - accuracy: 0.8119 - 

27/28 [===========================>..] - ETA: 0s - loss: 0.2171 - accuracy: 0.9233
Epoch 85: val_loss did not improve from 0.41966
28/28 [==============================] - 1s 46ms/step - loss: 0.2169 - accuracy: 0.9234 - val_loss: 0.4520 - val_accuracy: 0.8672
Epoch 86/300
27/28 [===========================>..] - ETA: 0s - loss: 0.2238 - accuracy: 0.9178
Epoch 86: val_loss did not improve from 0.41966
28/28 [==============================] - 1s 46ms/step - loss: 0.2235 - accuracy: 0.9177 - val_loss: 0.4768 - val_accuracy: 0.8592
Epoch 87/300
27/28 [===========================>..] - ETA: 0s - loss: 0.2176 - accuracy: 0.9275
Epoch 87: val_loss did not improve from 0.41966
28/28 [==============================] - 1s 45ms/step - loss: 0.2172 - accuracy: 0.9277 - val_loss: 0.4482 - val_accuracy: 0.8643
Epoch 88/300
27/28 [===========================>..] - ETA: 0s - loss: 0.2160 - accuracy: 0.9275
Epoch 88: val_loss did not improve from 0.41966
28/28 [==============================] - 1s 47m

28/28 [==============================] - ETA: 0s - loss: 0.1414 - accuracy: 0.9510
Epoch 143: val_loss did not improve from 0.30387
28/28 [==============================] - 1s 46ms/step - loss: 0.1414 - accuracy: 0.9510 - val_loss: 0.3776 - val_accuracy: 0.9015
Epoch 144/300
27/28 [===========================>..] - ETA: 0s - loss: 0.1309 - accuracy: 0.9520
Epoch 144: val_loss did not improve from 0.30387
28/28 [==============================] - 1s 46ms/step - loss: 0.1309 - accuracy: 0.9520 - val_loss: 0.3495 - val_accuracy: 0.9107
Epoch 145/300
27/28 [===========================>..] - ETA: 0s - loss: 0.1188 - accuracy: 0.9615
Epoch 145: val_loss did not improve from 0.30387
28/28 [==============================] - 1s 45ms/step - loss: 0.1190 - accuracy: 0.9613 - val_loss: 0.4141 - val_accuracy: 0.8924
Epoch 146/300
27/28 [===========================>..] - ETA: 0s - loss: 0.1259 - accuracy: 0.9566
Epoch 146: val_loss did not improve from 0.30387
28/28 [==============================] -

Epoch 173/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0958 - accuracy: 0.9669
Epoch 173: val_loss did not improve from 0.30387
28/28 [==============================] - 1s 46ms/step - loss: 0.0953 - accuracy: 0.9671 - val_loss: 0.4029 - val_accuracy: 0.9044
Epoch 174/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0978 - accuracy: 0.9648
Epoch 174: val_loss did not improve from 0.30387
28/28 [==============================] - 1s 46ms/step - loss: 0.0989 - accuracy: 0.9644 - val_loss: 0.4026 - val_accuracy: 0.9061
Epoch 175/300
27/28 [===========================>..] - ETA: 0s - loss: 0.1122 - accuracy: 0.9630
Epoch 175: val_loss did not improve from 0.30387
28/28 [==============================] - 1s 47ms/step - loss: 0.1123 - accuracy: 0.9626 - val_loss: 0.3772 - val_accuracy: 0.9073
Epoch 176/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0993 - accuracy: 0.9653
Epoch 176: val_loss did not improve from 0.30387
28/28 [===================

28/28 [==============================] - 1s 47ms/step - loss: 0.0737 - accuracy: 0.9752 - val_loss: 0.3385 - val_accuracy: 0.9227
Epoch 203/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0840 - accuracy: 0.9724
Epoch 203: val_loss did not improve from 0.29295
28/28 [==============================] - 1s 46ms/step - loss: 0.0837 - accuracy: 0.9724 - val_loss: 0.3217 - val_accuracy: 0.9216
Epoch 204/300
28/28 [==============================] - ETA: 0s - loss: 0.0863 - accuracy: 0.9719
Epoch 204: val_loss did not improve from 0.29295
28/28 [==============================] - 1s 47ms/step - loss: 0.0863 - accuracy: 0.9719 - val_loss: 0.3049 - val_accuracy: 0.9262
Epoch 205/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0830 - accuracy: 0.9718
Epoch 205: val_loss did not improve from 0.29295
28/28 [==============================] - 1s 48ms/step - loss: 0.0832 - accuracy: 0.9715 - val_loss: 0.3304 - val_accuracy: 0.9216
Epoch 206/300
27/28 [=====================

Epoch 262/300
28/28 [==============================] - ETA: 0s - loss: 0.0611 - accuracy: 0.9785
Epoch 262: val_loss did not improve from 0.27197
28/28 [==============================] - 1s 46ms/step - loss: 0.0611 - accuracy: 0.9785 - val_loss: 0.3091 - val_accuracy: 0.9410
Epoch 263/300
28/28 [==============================] - ETA: 0s - loss: 0.0678 - accuracy: 0.9767
Epoch 263: val_loss did not improve from 0.27197
28/28 [==============================] - 1s 47ms/step - loss: 0.0678 - accuracy: 0.9767 - val_loss: 0.3291 - val_accuracy: 0.9370
Epoch 264/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0661 - accuracy: 0.9782
Epoch 264: val_loss did not improve from 0.27197
28/28 [==============================] - 1s 46ms/step - loss: 0.0659 - accuracy: 0.9782 - val_loss: 0.3466 - val_accuracy: 0.9262
Epoch 265/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0565 - accuracy: 0.9809
Epoch 265: val_loss did not improve from 0.27197
28/28 [===================

Epoch 292/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0541 - accuracy: 0.9832
Epoch 292: val_loss did not improve from 0.27197
28/28 [==============================] - 1s 47ms/step - loss: 0.0538 - accuracy: 0.9834 - val_loss: 0.3809 - val_accuracy: 0.9290
Epoch 293/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0618 - accuracy: 0.9782
Epoch 293: val_loss did not improve from 0.27197
28/28 [==============================] - 1s 46ms/step - loss: 0.0626 - accuracy: 0.9782 - val_loss: 0.4351 - val_accuracy: 0.9067
Epoch 294/300
28/28 [==============================] - ETA: 0s - loss: 0.0638 - accuracy: 0.9777
Epoch 294: val_loss did not improve from 0.27197
28/28 [==============================] - 1s 47ms/step - loss: 0.0638 - accuracy: 0.9777 - val_loss: 0.3747 - val_accuracy: 0.9250
Epoch 295/300
27/28 [===========================>..] - ETA: 0s - loss: 0.0526 - accuracy: 0.9819
Epoch 295: val_loss did not improve from 0.27197
28/28 [===================

In [44]:
# Evaluating the model on the training and testing set
score = model_relu.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model_relu.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9948461055755615
Testing Accuracy:  0.9398969411849976


In [45]:
test_accuracy=model_relu.evaluate(x_test,y_test,verbose=0)
print(test_accuracy[1])

0.9398969411849976
